<a href="https://colab.research.google.com/github/MichelleFn/Finetuning-greek-bert-for-goverment-gazetes/blob/main/Binary_text_classification_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Installing libraries:**




In [1]:
!pip install unicodedata
!pip install transformers
!pip install torch
!pip install tensorflow
!pip install nlpaug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement unicodedata (from versions: none)
ERROR: No matching distribution found for unicodedata
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import unicodedata
from transformers import AutoTokenizer, TFAutoModel
import os, glob
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import tensorflow as tf
from keras import backend as K
from imblearn.over_sampling import RandomOverSampler
import nlpaug.augmenter.word as naw
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**Mounting csv file from google drive**

In [3]:
from google.colab import drive
drive.mount('/content/drive')
# folder_1='/content/drive/My Drive/csv/csv/dataset_1/' #first dataset 
folder_2='/content/drive/My Drive/csv/csv/dataset_2/' #second dataset


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Loading Bert tokenizer and TensorFlow model**

In [4]:
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/bert-base-greek-uncased-v1")
bert = TFAutoModel.from_pretrained("nlpaueb/bert-base-greek-uncased-v1",hidden_dropout_prob=0.5)
aug = naw.ContextualWordEmbsAug(model_path='nlpaueb/bert-base-greek-uncased-v1', action="substitute")

Some layers from the model checkpoint at nlpaueb/bert-base-greek-uncased-v1 were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at nlpaueb/bert-base-greek-uncased-v1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


**Stop words:**

In [5]:
stop_words  = nltk.corpus.stopwords.words('greek')


new_stopwords = ['της','τη','τους','ένας','ενός','ένα','μια','μιας','μιαν','αυτός','αυτή','αυτό','αυτοί', #adding extra stop words that weren't included
                 'αυτά','αυτών','ούτος','αυτούς','εκείνος','εκείνη','εκείνο','εκείνοι','εκείνων','εκείνα',
                 'ποιος','ποια','ποιοι','ποιων','ποιους','ποιες','πια','είμαι','είναι','είμαστε','είστε',
                 'εγώ','εσύ','εμείς','εσείς','άλλος','άλλη','άλλο','άλλων','άλλους','άλλα','κατ’','ως','ή',
                 'ούτε','ποτέ','πότε','προς','πρός','υπέρ','άμα','πέρι','οπως','όπως','από','ενώ','συν','πώς'
                 ,'εάν','προ','μη','ίσως','κάθε','καθε','ότι','ό,τι','όσο','στα','στους']

stop_words.extend(new_stopwords)
print(stop_words)



['αλλα', 'αν', 'αντι', 'απο', 'αυτα', 'αυτεσ', 'αυτη', 'αυτο', 'αυτοι', 'αυτοσ', 'αυτουσ', 'αυτων', 'αἱ', 'αἳ', 'αἵ', 'αὐτόσ', 'αὐτὸς', 'αὖ', 'γάρ', 'γα', 'γα^', 'γε', 'για', 'γοῦν', 'γὰρ', "δ'", 'δέ', 'δή', 'δαί', 'δαίσ', 'δαὶ', 'δαὶς', 'δε', 'δεν', "δι'", 'διά', 'διὰ', 'δὲ', 'δὴ', 'δ’', 'εαν', 'ειμαι', 'ειμαστε', 'ειναι', 'εισαι', 'ειστε', 'εκεινα', 'εκεινεσ', 'εκεινη', 'εκεινο', 'εκεινοι', 'εκεινοσ', 'εκεινουσ', 'εκεινων', 'ενω', 'επ', 'επι', 'εἰ', 'εἰμί', 'εἰμὶ', 'εἰς', 'εἰσ', 'εἴ', 'εἴμι', 'εἴτε', 'η', 'θα', 'ισωσ', 'κ', 'καί', 'καίτοι', 'καθ', 'και', 'κατ', 'κατά', 'κατα', 'κατὰ', 'καὶ', 'κι', 'κἀν', 'κἂν', 'μέν', 'μή', 'μήτε', 'μα', 'με', 'μεθ', 'μετ', 'μετά', 'μετα', 'μετὰ', 'μη', 'μην', 'μἐν', 'μὲν', 'μὴ', 'μὴν', 'να', 'ο', 'οι', 'ομωσ', 'οπωσ', 'οσο', 'οτι', 'οἱ', 'οἳ', 'οἷς', 'οὐ', 'οὐδ', 'οὐδέ', 'οὐδείσ', 'οὐδεὶς', 'οὐδὲ', 'οὐδὲν', 'οὐκ', 'οὐχ', 'οὐχὶ', 'οὓς', 'οὔτε', 'οὕτω', 'οὕτως', 'οὕτωσ', 'οὖν', 'οὗ', 'οὗτος', 'οὗτοσ', 'παρ', 'παρά', 'παρα', 'παρὰ', 'περί', 'περὶ', 'πο

**Class for reading csv files and creating datasets:**

**Text preprocess:**


In [6]:
def preprocess_data(df): #method that makes laters lower case,removes '/n',turns labels binary and removes stop words
      
  df["RawParagraph"] = df["RawParagraph"].str.lower() 
    
  df = df.replace(r'\n',' ', regex=True) 
      
  df['RespAPrediction'] = df['RespAPrediction'].replace(  #the extra variations of 'RespA' are because of unresolved typos in the certain csv files       
                                                  ['Non-RespA','RespA',' RespA','RespA '],
                                                  [0,1,1,1]).astype(int)

  df['RawParagraph'] = df['RawParagraph'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)])) 
  df = df.rename(columns={'RawParagraph': 'text','RespAPrediction': 'label'}) 
      
  return df

**Creating large dataset:**

In [7]:
def create_dataset(): 
     
  files = os.path.join(folder_2, "*.csv")
  files = glob.glob(files)
  temp_df = pd.concat(map(pd.read_csv, files), ignore_index=True)
  df=preprocess_data(temp_df)
        
  return df

**Creates table of csvs:**

In [8]:
def csv_dataset(): #creates a table where at each place there is a pandas dataframe for each csv
     
  dataframes_list = []
  list_of_names =['csva','csvb','csv0','csv1','csv2','csv3','csv4','csv5','csv6','csv7']
  col_list=['RawParagraph','RespAPrediction']
      
  for i in range(len(list_of_names)):

    temp_df = pd.read_csv(folder_2+list_of_names[i]+".csv",usecols=col_list)
    df=preprocess_data(temp_df)
    dataframes_list.append(df)

  return dataframes_list

In [9]:
dataframes_list=csv_dataset() #Calling the methods and creating a list 
text=dataframes_list[0]['text']

In [ ]:
display(dataframes_list[5])

**Encoding input text:**

In [11]:
def encode_bert(text): #encodes text to create BERT embeddings (input_ids, attention_masks)

  seq_len= 256
  num_samples=len(text)
  Xids=np.zeros((num_samples,seq_len))
  Xmask=np.zeros((num_samples,seq_len))
  
  for i,sentence in enumerate(text):
    
    tokens = tokenizer.encode_plus(sentence, max_length=seq_len, truncation=True, 
                                   padding='max_length',add_special_tokens=True,return_tensors='tf')
   
    Xids[i,:]=tokens['input_ids']
    Xmask[i,:]=tokens['attention_mask']


  
  inputs = {
      'input_ids': Xids,
      'attention_mask': Xmask
      }
  

  return inputs

**Creating data combinations:**

In [12]:
def create_dataset_combinations(fold_number,all_csvs): #creates dataset combination for k fold cross validation
  
  x_test=0
  y_test=0
  df= pd.DataFrame(columns=['text','label'])
  x_train=[]
  y_train=[]
  y=0
  special_tokens_train=[]

  print('FEK',fold_number,'used for testing, the rest for training')
  
  for csv in all_csvs: 
    
    if y==fold_number: #if the fold loop number is equal to special counter y, the csv is saved for the validation set
      x_test=csv['text'].values
      y_test=csv['label'].values
  
    else: #the rest of the csvs are joined together
      x_train.append(csv['text'].values)
      y_train.append(csv['label'].values)

    y=y+1
  
  x_train=np.concatenate(x_train, axis=0 )
  y_train=np.concatenate(y_train,axis=0)
  
  return x_test,y_test,x_train,y_train

**Creating model**:

In [13]:
def create_model(optimizer,loss,acc): 
  
  seq_len=256
  input_ids=tf.keras.layers.Input(shape=(seq_len,),name='input_ids',dtype='int32')
  mask=tf.keras.layers.Input(shape=(seq_len,),name='attention_mask',dtype='int32')

  embeddings=bert.bert(input_ids,attention_mask=mask)[1]
 
  x = tf.keras.layers.Dense(512,activation='relu')(embeddings)
  layer2 = tf.keras.layers.Dropout(0.4)(x)

  y = tf.keras.layers.Dense(2,activation='sigmoid',name='outputs')(layer2)
  model=tf.keras.Model(inputs=[input_ids,mask],outputs=y)
  model.compile(optimizer=optimizer, loss=loss, metrics=[acc])
  
  return model

In [14]:
optimizer= tf.keras.optimizers.Adam(learning_rate=0.0001,decay=1e-6)
loss=tf.keras.losses.BinaryCrossentropy()
acc=tf.keras.metrics.BinaryAccuracy('accuracy')


 **10 fold cross validation:**

In [15]:
def training_model(fold_i):
  
  i=fold_i #i=0-9 the loop doesn't work because the model doesn't get deleted so the collab Runtime has to be restarted

  x_val,y_val,x_train,y_train= create_dataset_combinations(i,dataframes_list)

  print('Size of training set: ',len(x_train))
  print('Size of test set: ', len(x_val))

  val_inputs=encode_bert(x_val) # return BERT embeddings
  val_labels=np.zeros((len(y_val),y_val.max()+1)) #on-hot encode labels
  val_labels[np.arange(len(y_val)),y_val] = 1
  
  train_inputs=encode_bert(x_train)
  train_labels=np.zeros((len(y_train),y_train.max()+1))
  train_labels[np.arange(len(y_train)),y_train] = 1
  

  model = create_model(optimizer,loss,acc)
  model.fit(train_inputs, train_labels,epochs=3,batch_size=16, validation_data=(val_inputs,val_labels))
  results=model.evaluate(val_inputs,val_labels)


In [ ]:
training_model(0) #enter manually 0-9

**Stratified 10 cross fold validation:**

In [ ]:
df=create_dataset() #only one big pandas dataset is needed for this version

In [ ]:
X_data=df['text']
Y_data=df['label']

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
#9 fold cross validation with complete dataset that is split into train and test each time
skf = StratifiedKFold(n_splits=10,shuffle=True, random_state=1)


a=0
score=[]
for train_index, test_index in skf.split(X_data, Y_data):
    
    print(f"Generating Inputs for fold {a}")
   
    
    
    train_X, test_X = X_data[train_index], X_data[test_index]
    train_y, test_y = Y_data[train_index], Y_data[test_index]

    train_inputs=encode_bert(train_X)

    
    train_label_array=train_y.values

    
    train_labels=np.zeros((len(train_y),train_label_array.max()+1))
    train_labels[np.arange(len(train_y)),train_label_array] = 1
    

    
    val_inputs=encode_bert(test_X)
    
    test_label_array=test_y.values
    test_labels=np.zeros((len(test_y),test_label_array.max()+1))
    test_labels[np.arange(len(test_y)),test_label_array] = 1
    
  
    model = create_model(optimizer,loss,acc)
    early_stopping=tf.keras.callbacks.EarlyStopping(patience=5)
    model.fit(train_inputs, train_labels,epochs=5,batch_size=8,
              validation_data=(val_inputs,test_labels),callbacks=early_stopping)
    results=model.evaluate(val_inputs,test_labels)
 
    a+=1


In [ ]:
#for testing purposes,only the first fek as testing and the rest for training
seq_len= 512
d3 = pd.concat([dataframes_list[0],dataframes_list[1],dataframes_list[2],dataframes_list[3],dataframes_list[4],dataframes_list[5],dataframes_list[6],dataframes_list[7]])
print(d3)
train_inputs=encode_bert(d3['text'])
print(train_inputs)
train_y=d3['label']

label_array=train_y.values
print(label_array)
len(label_array)

train_label_array=train_y
train_labels=np.zeros((len(train_y),train_label_array.max()+1))
train_labels[np.arange(len(train_y)),train_label_array] = 1
print(train_labels)
test_inputs=encode_bert(dataframes_list[8]['text'])
test_y=d3['label']
test_label_array=test_y
test_labels=np.zeros((len(test_y),test_label_array.max()+1))
test_labels[np.arange(len(test_y)),test_label_array] = 1

early_stopping=tf.keras.callbacks.EarlyStopping(patience=5)
model = create_model(optimizer,loss,acc)
model.fit(train_inputs, train_labels,epochs=5,batch_size=8,
              validation_data=(test_inputs,test_labels),callbacks=early_stopping)
results=model.evaluate(val_inputs,test_labels)
